<a href="https://colab.research.google.com/github/nicolafan/i-dream-my-painting2/blob/main/i_dream_my_painting_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 I Dream My Painting: Demo

**This Colab Notebook requires a GPU runtime. Change the runtime to use a T4 GPU.**

This notebook launches a demo of IDMP where you can use our pipeline to inpaint images interactively using Streamlit.

## Setup HF Token

You will need to download models from the Hugging Face Hub:

1. Make sure to have a [Hugging Face account](https://huggingface.co/join) and be logged in.
2. Log in locally by entering your [Hugging Face Hub access token](https://huggingface.co/docs/hub/security-tokens#what-are-user-access-tokens) below, which can be [found here](https://huggingface.co/settings/tokens)

In [ ]:
from huggingface_hub import login

token = ###### FILL ME!!! ######
login(token=token)

Now you can run all the cells below to start the demo... The process will require around ~10/15 minutes to install dependencies and download models.

## Install and Import Libraries

In [ ]:
!pip install -q gdown
!pip install -q streamlit
!pip install -q streamlit-drawable-canvas
!pip install -q bitsandbytes
!npm install -q localtunnel
!pip install -q diffusers==0.29.2

import gdown
import os
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from diffusers import StableDiffusionInpaintPipeline
import torch

## Download the Repository

In [ ]:
# Replace with your actual repository URL
repo_url = "https://github.com/cilabuniba/i-dream-my-painting.git"

# Clone the repository using the authenticated URL
!git clone {repo_url}

# Optionally, change the current directory to the cloned repository
# !cd YOUR_PRIVATE_REPO

## Download Stable Diffusion 2 Inpainting

In [ ]:
import huggingface_hub

huggingface_hub.snapshot_download(
    "stabilityai/stable-diffusion-2-inpainting",
    local_dir="/content/i-dream-my-painting/models/stable-diffusion-2-inpainting",
    ignore_patterns=["*.safetensors", "*.fp16.bin", "*model_index.json", "*unet/config.json"]
)

## Get Our Model Checkpoints

In [ ]:
id = "1_O7XTDjU-EgbjGwVT194NQJemdZXcrMe"
gdown.download_folder(id=id)

In [ ]:
!unzip /content/models/multimask.zip
!unzip /content/models/rca.zip

## Download LLaVA

In [ ]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-vicuna-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-vicuna-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)
del processor
del model
torch.cuda.empty_cache()

## Setup Paths

In [ ]:
os.rename("/content/i-dream-my-painting/models/sd_replacements/model_index.json", "/content/i-dream-my-painting/models/stable-diffusion-2-inpainting/model_index.json")
os.rename("/content/i-dream-my-painting/models/sd_replacements/config.json", "/content/i-dream-my-painting/models/stable-diffusion-2-inpainting/unet/config.json")

In [ ]:
os.rename("/content/multimask", "/content/i-dream-my-painting/models/llava/multimask")
os.rename("/content/rca", "/content/i-dream-my-painting/models/sd/rca")

## Download Example Images

In [ ]:
# prompt: download mona lisa, starry night and the birth of venus from wikiart and put them in /content/i-dream-my-painting/mm_inp_dataset/images
import os
import requests
from io import BytesIO
from PIL import Image

def download_image(image_url, image_path):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        image = Image.open(BytesIO(response.content))
        image.save(image_path)
        print(f"Downloaded: {image_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {image_url}: {e}")
    except Exception as e:
        print(f"Error processing {image_url}: {e}")


image_urls = {
    "Mona Lisa": "https://uploads7.wikiart.org/images/leonardo-da-vinci/mona-lisa.jpg!Large.jpg",
    "Starry Night": "https://uploads0.wikiart.org/00175/images/vincent-van-gogh/starry-night-over-the-rhone.jpg!Large.jpg",
    "The Birth of Venus": "https://uploads6.wikiart.org/images/sandro-botticelli/the-birth-of-venus-1485(1).jpg!Large.jpg"
}


output_dir = "/content/i-dream-my-painting/data/mm_inp_dataset/images"
os.makedirs(output_dir, exist_ok=True)

for image_name, image_url in image_urls.items():
    image_path = os.path.join(output_dir, f"{image_name.replace(' ', '_')}.jpg")
    download_image(image_url, image_path)

## Launch the Demo!

Here we are! Execute the following two cells to start the demo.

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Click on the produced URL and insert the above IP address in the page that opens to access the demo. In case of errors try to refresh the page, or try to access the URL using a different browser... Happy inpainting!

In [ ]:
os.chdir("/content/i-dream-my-painting")
!streamlit run inpainting/demo.py --server.fileWatcherType none &>/content/logs.txt &
!npx localtunnel --port 8501